## Data Injector

In [1]:
import cx_Oracle

In [2]:
import time
s = time.time()
connection = cx_Oracle.connect(
    user="sourceadmin",
    password="sourcepassword",
    dsn="oracledb-source.czpldjgtmbmx.us-west-2.rds.amazonaws.com/dbsource")
cursor = connection.cursor()
connection.close()
print(time.time()-s)

0.19763541221618652


```sql
CREATE TABLE  "TABLE1" (
    t1_pk CHAR(10) NOT NULL, 
    tag CHAR(40) NOT NULL, 
    number NUMBER(10,2) NOT NULL, 
    var VARCHAR2(17) NOT NULL,
    cdc VARCHAR2(25),
    CONSTRAINT TABLE1_CONSTRAINT PRIMARY KEY (t1_pk)
);

CREATE TABLE  "TABLE2"(
    t2_pk INT NOT NULL, 
    tag CHAR(40) NOT NULL, 
    number NUMBER(10,2) NOT NULL, 
    var VARCHAR2(17) NOT NULL,
    cdc VARCHAR2(25),
    refrence_t1 CHAR(6),
    CONSTRAINT TABLE2_CONSTRAINT FOREIGN KEY (refrence_t1) REFERENCES TABLE1(t1_pk) ON DELETE CASCADE
);

CREATE TABLE  "TABLE3"(
    tag CHAR(40) NOT NULL, 
    number NUMBER(10,2) NOT NULL, 
    var VARCHAR2(17) NOT NULL,
    cdc VARCHAR2(25)
);

CREATE TABLE  "CNT"(
    insert_t1 INT, 
    update_t1 INT,
    delete_t1 INT,
    block_t1 INT,
    insert_t2 INT, 
    update_t2 INT,
    delete_t2 INT,
    block_t2 INT,
    insert_t3 INT, 
    update_t3 INT,
    delete_t3 INT,
    block_t3 INT
);
```


In [1]:
import time
import cx_Oracle
from getpass import getpass
from tqdm import tqdm
import random

class DataGen:
    
    def __init__(self):
        self.host = None
        self.port = None
        self.user = None
        self.pasw= None
        self.sid = None
        self.connection = None
        self.cursor = None
    
    def connect(self, params = None):
        if params:
            print("[connection] Reading ORACLE connection parameters ...")
            self.host = params["host"]
            self.port = params["port"]
            self.sid  = params["sid"]
            self.user = params["user"]
            self.pasw = params["pass"]
        else:
            print("[connection] Please define ORACLE connection parameters")
            self.host = self._input("host",1)
            self.port = self._input("port",1)
            self.sid  = self._input("sid",1)
            self.user = self._input("user",1)
            self.pasw = self._input("pass",1,True)

        self.port = int(self.port)
        start = time.time()
        self.connection = cx_Oracle.connect(
            user=self.user,
            password=self.pasw,
            dsn=f"{self.host}/{self.sid}")
        self.cursor = self.connection.cursor()
        self.pasw = None
        dtime = time.time() - start
        print(f"[connection] Connected: elapsed time {dtime}s")
    
    def disconnect(self):
        print("[connection] Disconnecting ...")
        start = time.time()
        self.connection.close()
        dtime = time.time() - start
        print(f"[connection] Disconected: elapsed time {dtime}s")
    
    def create_tables(self, verbose = False):
        print("[tables] Creating tables ...")
        start = time.time()
        
        table1 = """CREATE TABLE "TABLE1" (
            "t1_pk" INT NOT NULL, 
            "block" INT NOT NULL, 
            "number" NUMBER(10,2) NOT NULL, 
            "var" VARCHAR2(17) NOT NULL,
            "cdc" VARCHAR2(25),
            CONSTRAINT "TABLE1_CONSTRAINT" PRIMARY KEY ("t1_pk")
        )"""
        table2 = """CREATE TABLE "TABLE2"(
            "t2_pk" INT NOT NULL, 
            "block" INT NOT NULL, 
            "number" NUMBER(10,2) NOT NULL, 
            "var" VARCHAR2(17) NOT NULL,
            "cdc" VARCHAR2(25),
            "refrence_t1" INT,
            CONSTRAINT "TABLE2_CONSTRAINT" FOREIGN KEY ("refrence_t1") REFERENCES TABLE1("t1_pk") ON DELETE CASCADE
        )"""
        table3 = """CREATE TABLE "TABLE3"(
            "block" INT NOT NULL, 
            "number" NUMBER(10,2) NOT NULL, 
            "var" VARCHAR2(17) NOT NULL,
            "cdc" VARCHAR2(25)
        )"""
        plsql_config = """
            begin
                rdsadmin.rdsadmin_util.force_logging(p_enable => true);
                rdsadmin.rdsadmin_util.alter_supplemental_logging(p_action => 'ADD');
                rdsadmin.rdsadmin_util.alter_supplemental_logging(p_action => 'ADD',p_type => 'ALL');
                rdsadmin.rdsadmin_util.alter_supplemental_logging(p_action => 'ADD',p_type => 'PRIMARY KEY');
                rdsadmin.rdsadmin_util.switch_logfile;
                rdsadmin.rdsadmin_master_util.create_archivelog_dir;
                rdsadmin.rdsadmin_master_util.create_onlinelog_dir;
            end;
        """
        
        self.sql(table1, "tables", "TABLE1 created", "TABLE1 warning", verbose)
        self.sql(table2, "tables", "TABLE2 created", "TABLE2 warning", verbose)
        self.sql(table3, "tables", "TABLE3 created", "TABLE3 warning", verbose)
        self.sql(plsql_config, "table", "PL/SQL Executed", "PL/SQL warning", verbose)
        self.connection.commit()
        
        dtime = time.time() - start
        print(f"[tables] Tables created: elapsed time {dtime}s")
    
    def drop_tables(self, verbose=False):
        print("[tables] Dropping tables ...")
        start = time.time()
        self.sql("DROP TABLE TABLE3", "tables", "TABLE3 droped", "TABLE3 warning", verbose)
        self.sql("DROP TABLE TABLE2", "tables", "TABLE2 droped", "TABLE2 warning", verbose)
        self.sql("DROP TABLE TABLE1", "tables", "TABLE1 droped", "TABLE1 warning", verbose)        
        self.connection.commit()
        dtime = time.time() - start
        print(f"[tables] Tables dropped: elapsed time {dtime}s")
    
    def insert_data(self):
        print("------------------")
        print("[data] INSERT")
        print("[data] List of tables")
        print("         [0] ALL TABLES")
        print("         [1] TABLE1")
        print("         [2] TABLE2")
        print("         [3] TABLE3")
        time.sleep(0.01)
        table = self._input("1. Enter the number of the table              [default:   0]",1)
        nrows = self._input("2. Number of new rows   [valid range 5 - 1000, default:  10]",1)
        delay = self._input("3. Cycle delay-ms       [valid range 0 - 1000, default:   0]",1)
        blckr = self._input("4. List of blocks to insert (example: 2,3,5)  [default: 1,2]",1)
        
        table = table.replace(" ","")
        nrows = nrows.replace(" ","")
        delay = delay.replace(" ","")
        blckr = blckr.replace(" ","")
        
        table = [1,2,3] if table == "" or table == "0" else [table]
        nrows = 10 if nrows == "" else nrows
        delay = 0 if delay == "" else delay
        blckr = "1,2" if blckr == "" else blckr
            
        try: 
            table = list(map(lambda x: int(x), table))
            nrows = int(nrows)
            delay = int(delay)
            
            for i in table:
                assert 0 <= i <= 3
        except:
            print("[data] Invalid character. Please, check inputs '1', '2', '3' !")
            return 

        try:
            blckrs = blckr.split(",")
            assert len(blckrs) > 0
            blckrs = list(map(lambda x: int(x),blckrs))
        except:
            print("[data] Invalid range. Please, check input '4'!")
            print("[data] Remember that the format for input '4' is 'numA,numB'. Example: 2,5'")
            return
            
        try:
            assert 5 <= nrows <= 1000
            assert 0 <= delay <= 1000
            delay = delay/1000
        except:
            print("[data] Invalid range !")
            return

        datos_t1 = []
        datos_t2 = []
        datos_t3 = []
        pks = []
        print("[data] Inserting data ...")  
        time.sleep(0.01)  
        start = time.time()
        current_table=0
        try:
            print("table:",table)
            if 1 in table:
                print("[data] TABLE1 ...")  
                current_table=1
                insert_t1 = list(self.cursor.execute("""SELECT MAX("t1_pk") FROM TABLE1"""))[0][0]
                insert_t1 = 0 if not insert_t1 else insert_t1 + 1
                for i in tqdm(range(nrows)):
                    d = (insert_t1 + i,
                         blckrs[random.randint(0,len(blckrs)-1)],
                         random.randint(0,1000),
                         str(hex(random.randint(0,1000))[2:]),
                         str(0)
                    )
                    self.cursor.execute(f"""INSERT INTO TABLE1 VALUES {d}""")
                    self.connection.commit()
                    pks.append(str(insert_t1 + i))
                    time.sleep(delay)

            if 2 in table:
                print("[data] TABLE2 ...") 
                current_table=2
                stored_pks = list(self.cursor.execute("""SELECT "t1_pk" FROM TABLE1"""))                
                stored_pks = list(map(lambda x: x[0], stored_pks))
                pks = list(set(pks + stored_pks))
                assert len(pks) > 0, "No primary keys found in TABLE1"
                
                insert_t2 = list(self.cursor.execute("""SELECT MAX("t2_pk") FROM TABLE2"""))[0][0]
                insert_t2 = 0 if not insert_t2 else insert_t2 + 1
                for i in tqdm(range(nrows)):
                    d = (insert_t2 + i,
                         blckrs[random.randint(0,len(blckrs)-1)],
                         random.randint(0,1000),
                         str(hex(random.randint(0,1000))[2:]),
                         str(0),
                         pks[random.randint(0,len(pks)-1)]
                    )
                    self.cursor.execute(f"""INSERT INTO TABLE2 VALUES {d}""")
                    self.connection.commit()
                    time.sleep(delay)
 
            if 3 in table:
                print("[data] TABLE3 ...") 
                current_table=3
                for i in tqdm(range(nrows)):
                    d = (blckrs[random.randint(0,len(blckrs)-1)],
                         random.randint(0,1000),
                         str(hex(random.randint(0,1000))[2:]),
                         str(0)
                    )
                    self.cursor.execute(f"""INSERT INTO TABLE3 VALUES {d}""")
                    self.connection.commit()
                    time.sleep(delay)
            
        except Exception as e:
            print(f"[data] Fatal error while insertign data in table {current_table}: \n{e}") 
            return
        
        dtime = time.time() - start
        print(f"[data] Inserts completed: elapsed time {dtime}s")
        
    def update_data(self):
        print("------------------")
        print("[data] UPDATE")
        print("[data] List of tables")
        print("         [1] TABLE1")
        print("         [2] TABLE2")
        print("         [3] TABLE3")
        time.sleep(0.01)
        table = self._input("1. Enter the number of the table                             [default: all]",1)
        blckr = self._input("2. List of blocks to update (example: 2,3,5 or -1 to update all) [required]",1)
        cdctx = self._input("3. Enter some text to identify changes in the DB        [max 20 characters]",1)
        if len(cdctx)>20:
            print("  The text is too long, only the first 20 character will be used!")
        
        table = table.replace(" ","")
        blckr = blckr.replace(" ","")
        cdctx = cdctx[:20]
        
        table = [1,2,3] if table == "" or table == "0" else [table]
        try: 
            table = list(map(lambda x: int(x), table))
        except:
            print("[data] Invalid character. Please, check input '1' !")
            return 
        try:
            blckrs = blckr.split(",")
            assert len(blckrs) > 0
            blckrs = list(map(lambda x: str(int(x)),blckrs))
            blckrs = ",".join(blckrs)
            print(f":{blckrs}")
        except:
            print("[data] Invalid range. Please, check input '2'!")
            print("[data] Remember that the format for input '2'. Example: 2,5'")
            return

        print("[data] Updating data ...") 
        time.sleep(0.01)
        start = time.time()
        try:
            for t in table:
                if len(blckrs)==1 and blckrs[0]==-1:
                    self.cursor.execute(f"""UPDATE TABLE{t} SET "cdc"='{cdctx}'""")
                else:
                    self.cursor.execute(f"""UPDATE TABLE{t} SET "cdc"='{cdctx}' WHERE "block" IN ({blckrs})""")
                self.connection.commit()
        except Exception as e:
            print(f"[data] Fatal error while updating data: \n{e}") 
            return
        dtime = time.time() - start
        print(f"[data] Update completed: elapsed time {dtime}s")
        
    def delete_data(self):
        print("------------------")
        print("[data] DELETE")
        print("[data] List of tables")
        print("         [1] TABLE1")
        print("         [2] TABLE2")
        print("         [3] TABLE3")
        time.sleep(0.01)
        table = self._input("1. Enter the number of the table                               [default: 1]",1)
        blckr = self._input("2. List of blocks to delete (example: 2,3,5 or -1 to delete all) [required]",1)
        
        table = table.replace(" ","")
        blckr = blckr.replace(" ","")
        
        table = [1,2,3] if table == "" or table == "0" else [table]
        try: 
            table = list(map(lambda x: int(x), table))
        except:
            print("[data] Invalid character. Please, check input '1' !")
            return 
        try:
            blckrs = blckr.split(",")
            assert len(blckrs) > 0
            blckrs = list(map(lambda x: str(int(x)),blckrs))
            blckrs = ",".join(blckrs)
            print(f":{blckrs}")
        except:
            print("[data] Invalid range. Please, check input '2'!")
            print("[data] Remember that the format for input '2'. Example: 2,5'")
            return

        print("[data] Deleting data ...") 
        time.sleep(0.01)
        start = time.time()
        try:
            for t in table:
                if len(blckrs)==1 and blckrs[0]==-1:
                    self.cursor.execute(f"""DELETE TABLE{t} CNT""")
                else:
                    self.cursor.execute(f"""DELETE TABLE{t} CNT WHERE "block" IN ({blckrs})""")
                self.connection.commit()
        except Exception as e:
            print(f"[data] Fatal error while deleting data: \n{e}")
            return
        dtime = time.time() - start
        print(f"[data] Delete completed: elapsed time {dtime}s")
    
    def list_blocks(self):
        print("------------------")
        print("[data] DELETE")
        print("[data] List of blocks")
    
    def dbinfo(self):
        pass
    
    def _input(self, msn, tabs=0, password=False):
        t = ''.join(["  "]*tabs)
        if password:
            return getpass(f"{t}{msn}:") 
        return input(f"{t}{msn}: ").strip()
    
    def sql(self, sql, command, msn, msnerr, verbose=False):
        try:
            self.cursor.execute(sql)
            print(f"[{command}] {msn}")
        except Exception as e:
            if verbose:
                print(f"[{command}] {msnerr}: \n{e}")

In [2]:
con = {
    "host": "oracledb-source.czpldjgtmbmx.us-west-2.rds.amazonaws.com",
    "port": "1521",
    "sid": "DBSOURCE",
    "user": "sourceadmin",
    "pass": "sourcepassword"
}

dgen = DataGen()
dgen.connect(con)

[connection] Reading ORACLE connection parameters ...
[connection] Connected: elapsed time 0.047097206115722656s


In [11]:
dgen.create_tables()

[tables] Creating tables ...
[tables] TABLE1 created
[tables] TABLE2 created
[tables] TABLE3 created
[tables] Tables created: elapsed time 0.028804540634155273s


In [10]:
dgen.drop_tables(True)

[tables] Dropping tables ...
[tables] TABLE3 droped
[tables] TABLE2 droped
[tables] TABLE1 droped
[tables] Tables dropped: elapsed time 0.0441431999206543s


In [12]:
dgen.insert_data()

------------------
[data] INSERT
[data] List of tables
         [0] ALL TABLES
         [1] TABLE1
         [2] TABLE2
         [3] TABLE3
  1. Enter the number of the table              [default:   0]: 
  2. Number of new rows   [valid range 5 - 1000, default:  10]: 100
  3. Cycle delay-ms       [valid range 0 - 1000, default:   0]: 
  4. List of blocks to insert (example: 2,3,5)  [default: 1,2]: 1,2,3,4,5


 51%|█████     | 51/100 [00:00<00:00, 501.10it/s]

[data] Inserting data ...
table: [1, 2, 3]
[data] TABLE1 ...


100%|██████████| 100/100 [00:00<00:00, 521.21it/s]

[data] TABLE2 ...



100%|██████████| 100/100 [00:00<00:00, 527.91it/s]

[data] TABLE3 ...
[data] Inserts completed: elapsed time 0.6106641292572021s


In [3]:
dgen.update_data()

------------------
[data] UPDATE
[data] List of tables
         [1] TABLE1
         [2] TABLE2
         [3] TABLE3
  1. Enter the number of the table                             [default: all]: 2
  2. List of blocks to update (example: 2,3,5 or -1 to update all) [required]: 1
  3. Enter some text to identify changes in the DB        [max 20 characters]: asdfsdfsdf
:1
[data] Updating data ...
[data] Update completed: elapsed time 0.006013154983520508s


In [4]:
dgen.delete_data()

------------------
[data] DELETE
[data] List of tables
         [1] TABLE1
         [2] TABLE2
         [3] TABLE3
  1. Enter the number of the table                               [default: 1]: 1
  2. List of blocks to delete (example: 2,3,5 or -1 to delete all) [required]: 1
:1
[data] Deleting data ...
[data] Delete completed: elapsed time 0.007298946380615234s


In [18]:
import configparser 

config = configparser.ConfigParser()
config.read('confs.conf')
config.has_section("DEFAULT")

False

In [11]:
config['DEFAULT']['ServerAliveInterval']

KeyError: 'ServerAliveInterval'

In [5]:
ss.

ValueError: I/O operation on closed file.